<a href="https://colab.research.google.com/github/TejSuklikar/GPT-3-Research-Project/blob/main/TQA_GPT3_Code_No_Context_and_With_Context_Test_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load the JSON File**

In [ ]:
#Library
import json
import pandas as pd

x = pd.read_json("/content/tqa_v2_test.json")

Create a Content Table where each row is either a Topic or Adjunct Topic for a given lesson.

In [ ]:
from pandas.core.apply import Apply
topics = x['topics']
adjunctTopics = x['adjunctTopics']

length = x.index.stop
i=0
contentTable = []
while i < length:
  for key in topics[i].keys():
    contentTable.append( [x['globalID'][i], x['lessonName'][i], "Topic", "", key, topics[i][key]['content']['text']])
    j = 0
    for key in adjunctTopics[i].keys():
      contentID = "A_" + str(i*1000+j)
      if (key != 'Vocabulary'):
        contentTable.append( [x['globalID'][i], x['lessonName'][i], "Adjunct Topic", str(key), contentID, adjunctTopics[i][str(key)]['content']['text']])
      j+=1
  i+=1


Create a simpler Content Table with just Topic content

In [ ]:
from pandas.core.apply import Apply
topics = x['topics']

length = x.index.stop
i=0
contentqaTable = []
while i < length:
  for key in topics[i].keys():
    content = topics[i][key]['content']['text']
    contentqaTable.append([x['globalID'][i], content])
  i+=1


Convert the content table to a Data Frame.

In [ ]:
ct = pd.DataFrame(contentqaTable,columns=['id','context',])

Since there are multiple rows of content for each Lesson ID, get the unique Lesson IDs. This will be useful in constructing Prompts per lesson

In [ ]:
lessonIds = ct.id.unique()


Create a consolidated Lesson Table with one row per Lesson, and all the associated content

In [ ]:
consolidatedLessonTable = []
for l in lessonIds:
  lessonContext = ""
  lessonContents = ct[ct.id == l]
  for index, row in lessonContents.iterrows():
    lessonContext += row['context'] + "\n"
  consolidatedLessonTable.append([l,lessonContext])
clt = pd.DataFrame(consolidatedLessonTable,columns=['id','content'])

Create a Question Answer Table with the Questions, Answer Choices, and Correct Answer per row. Additionally the associated Lesson ID is also stored for looking up and joining to the Lesson Table content.

In [ ]:
questions = x['questions']
length = x.index.stop
i=0

questionAnswerTable = []
answerTable =[]
while i < length:
  for key in questions[i]['nonDiagramQuestions'].keys():
    lessonID = x['globalID'][i]
    lessonName = x['lessonName'][i]
    questionID = key
    questionText = questions[i]['nonDiagramQuestions'][key]['beingAsked']['processedText']
    questionType = questions[i]['nonDiagramQuestions'][key]['questionType']
    questionSubType = questions[i]['nonDiagramQuestions'][key]['questionSubType']
    correctAnswerChoice = questions[i]['nonDiagramQuestions'][key]['correctAnswer']['processedText']
    answerChoices = questions[i]['nonDiagramQuestions'][key]['answerChoices']
    answerChoicesPrompt = ""
    correctAnswerDetail = ""
    #answer_start = content.find(correctAnswerChoice)
    for key2 in questions[i]['nonDiagramQuestions'][key]['answerChoices'].keys():
      answerChoicesPrompt = answerChoicesPrompt + questions[i]['nonDiagramQuestions'][key]['answerChoices'][key2]['rawText'] + "; "

    answerChoicesPrompt = answerChoicesPrompt[:-2]  
    questionAnswerTable.append([lessonID,questionText,correctAnswerChoice, answerChoicesPrompt])
  i+=1

In [ ]:
qat = pd.DataFrame(questionAnswerTable,columns=['id','question', 'correct answer', 'answer choices'])

In [ ]:
len(qat)

2512

In [ ]:
qat.to_csv("qat_test.csv")

For the No Context Learning experiment, build a Prompt Table with just the prompt, Question plus Answer Choices. One Lesson per Row.

For the Lesson Context Learning experiment, build a Prompt Table that combines the Lesson Content and Question plus Answer Choices in a single string. One Lesson per Row. 

Since the prompt needs to be less than 2,048 tokens, limit to only those lesson where the associatated prompts are less than 8,500 characters (discovered by trial and error using the tokenizer tool at https://beta.openai.com/tokenizer)


In [ ]:
lessonContextPromptInstructions = "Use the lesson text below to answer the following questions by picking one of the choices provided. Only include the letter of the answer choice listed. For example, 3.c.\n\n"
noContextPromptInstructions = "Answer the following questions by picking one of the choices provided. Only include the letter of the answer choice listed.\n\n"

lessonContextPromptTable = []
noContextPromptTable = []

lessonContextAnswerKeyTable = []
noContextAnswerKeyTable = []

for i,l in clt.iterrows():
  lcPrompt = lessonContextPromptInstructions + "Lesson:\n" + l['content'] +"\n\n"+"Questions:\n"
  ncPrompt = noContextPromptInstructions + "Questions:\n"  
  lcAnswers = ""
  lessonQAT = qat[qat.id == l['id']]
  qnum = 1
  for index,row in lessonQAT.iterrows():
    lcPrompt = lcPrompt + str(qnum) + ". " + row['question'] +"\n" + row['answer choices'] + "\n\n"
    ncPrompt = ncPrompt + str(qnum) + ". " + row['question'] +"\n" + row['answer choices'] + "\n\n"
    lcAnswers = lcAnswers + str(qnum) + ". " + row['correct answer'] + "; "
    noContextAnswerKeyTable.append([l['id'],qnum,row['correct answer']])
    qnum += 1
      
  lcAnswers = lcAnswers[:-2]

  for i in range(qnum-1):
    lcPrompt = lcPrompt + str(i+1) + ". ?\n"
    ncPrompt = ncPrompt + str(i+1) + ". ?\n"

  lcPrompt = lcPrompt + "===="
  ncPrompt = ncPrompt + "===="
 
 # Since the prompt needs to be less than 2,048 tokens, limit to only those prompts that are less than 8,500 characters (discovered by trial and error using the tokenizer tool at https://beta.openai.com/tokenizer)
  if (len(lcPrompt)<15000):
    noContextPromptTable.append ([l['id'],ncPrompt])
    lessonContextPromptTable.append ([l['id'],lcPrompt])
    lessonContextAnswerKeyTable.append ([l['id'],lcAnswers])





In [ ]:
lcpt = pd.DataFrame(lessonContextPromptTable,columns=['Lesson ID','Prompt'])
lcakt = pd.DataFrame(lessonContextAnswerKeyTable,columns=['Lesson ID','Answers'])
ncpt = pd.DataFrame(noContextPromptTable,columns=['Lesson ID','Prompt'])
ncakt = pd.DataFrame(noContextAnswerKeyTable,columns=['Lesson ID','Question Number','Answer'])

In [ ]:
len(lcakt)

208

In [ ]:
lcpt['prompt length'] = lcpt.apply(lambda x: len(str(x['Prompt'])), axis=1)

In [ ]:
lcpt.sort_values(by=['prompt length'], ascending=False)

,Lesson ID,Prompt,prompt length
1,L_0013,Use the lesson text below to answer the follow...,14575
175,L_0730,Use the lesson text below to answer the follow...,14505
58,L_0382,Use the lesson text below to answer the follow...,14078
173,L_0712,Use the lesson text below to answer the follow...,13928
46,L_0365,Use the lesson text below to answer the follow...,13747
...,...,...,...
112,L_0565,Use the lesson text below to answer the follow...,2095
203,L_1020,Use the lesson text below to answer the follow...,2054
206,L_1046,Use the lesson text below to answer the follow...,1910
183,L_0826,Use the lesson text below to answer the follow...,1864


In [ ]:
lcpt.head(5)

,Lesson ID,Prompt,prompt length
0,L_0001,Use the lesson text below to answer the follow...,18705
1,L_0005,Use the lesson text below to answer the follow...,6672
2,L_0013,Use the lesson text below to answer the follow...,14575
3,L_0024,Use the lesson text below to answer the follow...,8788
4,L_0026,Use the lesson text below to answer the follow...,9172


In [ ]:
lcpt.to_csv("lcpt_test.csv")

In [ ]:
lcakt.to_csv("lcakt_test.csv")
ncpt.to_csv("ncpt_test.csv")
ncakt.to_csv("ncakt_test.csv")

In [ ]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 56.9 MB/s 
  Created wheel for openai: filename=openai-0.23.0-py3-none-any.whl size=54478 sha256=e6f899b3c52c2bb60951248c6bfad02c344c3798c5fc77e16af2718e8093d2f6
  Stored in directory: /root/.cache/pip/wheels/70/d5/31/f9f67660319d89e4f54501d27b1e90f88a3309c42ea4fd734c
Successfully built openai


In [ ]:

OPENAI_API_KEY="Your API Key"

In [ ]:
def response_to_table (lId, r, answer_table):
  answer_list = r.strip().split("\n")
  for i in answer_list:
    row = i.split(".")
    answer_table.append([lId, row[0],row[1].strip()])
  return answer_table

In [ ]:
def lesson_answer (lId,p,answerTable):
  import os
  import openai

  openai.api_key = OPENAI_API_KEY

  start_sequence = "\nA:"
  restart_sequence = "\n\nQ: "

  response = openai.Completion.create(
    model="text-davinci-002",
    prompt=p,
    temperature=0,
    max_tokens=200,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["===="]
  )
 # if response['choices'][0]['finish_reason']=='stop':
  answerTable = response_to_table (lId, response['choices'][0]['text'],answerTable)
  
  return answerTable

Test it out with one lesson

In [ ]:
import os
import openai

openai.api_key = OPENAI_API_KEY

start_sequence = "\nA:"
restart_sequence = "\n\nQ: "

response = openai.Completion.create(
    model="text-davinci-002",
    prompt=lcpt['Prompt'][175],
    temperature=0,
    max_tokens=200,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["===="]
  )

In [ ]:
print (response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n1. b\n2. a\n3. b\n4. d\n5. c\n6. d\n7. a\n8. b\n9. b\n10. c\n11. a\n12. a\n13. b\n14. b\n15. a\n16. a\n17. b\n18. a\n19. a\n20. a\n21. b\n22. c\n23. a\n24. b\n25. a\n26. a\n27. b\n28. c\n29. d\n30. g\n31. b\n32. e\n33. f\n34. a"
    }
  ],
  "created": 1663524803,
  "id": "cmpl-5rt59Y0FoueOycvOu1Sf6rx3glWzt",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 137,
    "prompt_tokens": 3654,
    "total_tokens": 3791
  }
}


Run the experiment with Lesson Context, 50 prompts at a time

In [ ]:
lcResultsTable = []

In [ ]:
print (lcpt.iloc[49])

Lesson ID                                                   L_0368
Prompt           Use the lesson text below to answer the follow...
prompt length                                                12744
Name: 49, dtype: object


In [ ]:
subsetTable = lcpt.iloc[38:49]
for row in subsetTable.itertuples():
  lcResultsTable = lesson_answer(row[1],row[2], lcResultsTable)


IndexError: ignored

In [ ]:
print(len(lcResultsTable))

681


In [ ]:
print(lcResultsTable[680])

['L_0365', 'a', 'stoneworts; b']


In [ ]:
ncResultsTable = []

In [ ]:
subsetTable = ncpt.iloc[36:37]
for row in subsetTable.itertuples():
  ncResultsTable = lesson_answer(row[1],row[2], ncResultsTable)

IndexError: ignored

In [ ]:
print(len(ncResultsTable),
len(ncakt))

1407 2512


In [ ]:
print (ncResultsTable[1226])

['L_0699', '7', 'b']


In [ ]:
print (ncResultsTable[1406])

['L_1075', '2', 'd']


In [ ]:
lcrat = pd.DataFrame(lcResultsTable,columns=['Lesson ID','Question Number','Returned Answer'])

In [ ]:
lcrat.iloc[114]

Lesson ID            L_0149
Question Number           1
Returned Answer    a) True;
Name: 114, dtype: object

In [ ]:
lcrat.to_csv("lcrat.csv")

In [ ]:
ncrat = pd.DataFrame(ncResultsTable,columns=['Lesson ID','Question Number','Returned Answer'] )

In [ ]:
convert_dict = {'Question Number': int}
ncrat = ncrat.astype(convert_dict)

In [ ]:
lcrat_clean = lcrat.drop_duplicates(subset = ['Lesson ID','Question Number'],keep='first').reset_index(drop=True)

In [ ]:
#fsrat_clean = fsrat_clean[fsrat_clean['Lesson ID'] != 'L_0886']

In [ ]:
convert_dict = {'Question Number': int}
lcrat_clean = lcrat_clean.astype(convert_dict)

In [ ]:
lcrat_clean['Returned Answer'] = lcrat_clean.apply(lambda x:  x['Returned Answer'][0], axis=1)

In [ ]:
lcrat_clean.iloc[114]

Lesson ID          L_0149
Question Number         1
Returned Answer         a
Name: 114, dtype: object

In [ ]:
zsrat_clean = pd.read_csv("/content/zs_answers_compared.csv")
zsrat = zsrat_clean.drop(['Unnamed: 0','Answer','Is Correct'],axis=1)

In [ ]:
ncrat['Returned Answer'] = ncrat.apply(lambda x: 'a' if x['Returned Answer'].lower() == 'true' else 'b' if x['Returned Answer'].lower() == 'false' else x['Returned Answer'], axis=1)

In [ ]:
ncrat.iloc[114]

Lesson ID          L_0445
Question Number         2
Returned Answer         a
Name: 574, dtype: object

In [ ]:
ncrat['Returned Answer'] = ncrat.apply(lambda x:  x['Returned Answer'][0], axis=1)

In [ ]:
lcrat_clean.dtypes

Lesson ID          object
Question Number     int64
Returned Answer    object
dtype: object

In [ ]:
lcCompt = lcrat_clean.merge (ncakt,how='inner',left_on=['Lesson ID','Question Number'], right_on=['Lesson ID','Question Number'])

In [ ]:
ncCompt = ncrat.merge (ncakt,how='inner',left_on=['Lesson ID','Question Number'], right_on=['Lesson ID','Question Number'])

In [ ]:
lcCompt.head(10)

,Lesson ID,Question Number,Returned Answer,Answer
0,L_0005,1,b,b
1,L_0005,2,b,b
2,L_0005,3,a,a
3,L_0005,4,b,b
4,L_0005,5,b,b
5,L_0005,6,a,a
6,L_0005,7,b,b
7,L_0005,8,b,b
8,L_0005,9,a,a
9,L_0005,10,a,a


In [ ]:
lcCompt['Is Correct'] = lcCompt.apply(lambda x: x['Answer'] == x['Returned Answer'], axis=1)

In [ ]:
ncCompt['Is Correct'] = ncCompt.apply(lambda x: x['Answer'] == x['Returned Answer'], axis=1)

In [ ]:
zsCompt.head(20)

Zero Shot Accuracy

In [ ]:
len(ncCompt[ncCompt['Is Correct']==True])/len(ncCompt)

0.7057569296375267

In [ ]:
compt.to_csv("fs_answers_compared.csv")
zsCompt.to_csv("zs_answers_compared.csv")

In [ ]:
fsCompt = pd.read_csv("/content/fs_answers_compared_fixed_playground.csv")

Few Shot Accuracy

In [ ]:
len(lcCompt[lcCompt['Is Correct']==1])/len(lcCompt)

0.8179775280898877

In [ ]:
zsCompt.head(10)

,Lesson ID,Question Number,Returned Answer,Answer,Is Correct
0,L_0007,1,d,b,False
1,L_0007,2,d,d,True
2,L_0007,3,d,c,False
3,L_0007,4,c,c,True
4,L_0007,5,d,a,False
5,L_0007,6,d,d,True
6,L_0007,7,d,d,True
7,L_0007,8,c,c,True
8,L_0007,9,c,c,True
9,L_0007,10,a,b,False


In [ ]:
combined_results_table = zsCompt.merge (fsCompt,how='inner',left_on=['Lesson ID','Question Number'], right_on=['Lesson ID','Question Number'],suffixes=('_zs','_fs'))

In [ ]:
combined_results_table.head(10)

,Lesson ID,Question Number,Returned Answer_zs,Answer_zs,Is Correct_zs,Returned Answer_fs,Answer_fs,Is Correct_fs
0,L_0007,1,d,b,False,d,b,False
1,L_0007,2,d,d,True,d,d,True
2,L_0007,3,d,c,False,d,c,False
3,L_0007,4,c,c,True,c,c,True
4,L_0007,5,d,a,False,a,a,True
5,L_0007,6,d,d,True,d,d,True
6,L_0007,7,d,d,True,d,d,True
7,L_0007,8,c,c,True,c,c,True
8,L_0007,9,c,c,True,c,c,True
9,L_0007,10,a,b,False,b,b,True


In [ ]:
combined_results_table.nunique()

Lesson ID             179
Question Number        34
Returned Answer_zs     11
Answer_zs               7
Is Correct_zs           2
Returned Answer_fs     10
Answer_fs               7
Is Correct_fs           2
dtype: int64

In [ ]:
len(combined_results_table)

2254

**Comparison of Accuracy for the common questions**

In [ ]:
print("Zero Shot Accuracy: " + str(len(combined_results_table[combined_results_table['Is Correct_zs']==True])/len(combined_results_table)) + "  Few Shot Accuracy: " + str(len(combined_results_table[combined_results_table['Is Correct_fs']==True])/len(combined_results_table)))

Zero Shot Accuracy: 0.7275953859804791  Few Shot Accuracy: 0.8478260869565217


In [ ]:
combined_results_table.to_csv("combined_results_final.csv")